## Importing packages

As always we start by importing necessary packages.

In [1]:
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scipy
import pandas as pd
import numpy as np
import random
from scipy.sparse.linalg import svds
random.seed(100)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Objective

We are given a dataset of moview ratings: (sometimes needs to be run again)

In [2]:
data = pd.read_csv('user_train_df.csv')
print(data.shape)
data.head()

(32550, 30)


,User ID,Item ID,Rating,timestamp,Age,Gender,Occupation,zip code,Movie Title,Release Date,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,168,5,874965478,24,M,technician,85711,Monty Python and the Holy Grail (1974),01-Jan-1974,...,0,0.0,0,0,0.0,0,0,0.0,0,0
1,1,172,5,874965478,24,M,technician,85711,"Empire Strikes Back, The (1980)",01-Jan-1980,...,0,0.0,0,0,0.0,1,1,0.0,1,0
2,1,165,5,874965518,24,M,technician,85711,Jean de Florette (1986),01-Jan-1986,...,0,0.0,0,0,0.0,0,0,0.0,0,0
3,1,156,4,874965556,24,M,technician,85711,Reservoir Dogs (1992),01-Jan-1992,...,0,0.0,0,0,0.0,0,0,1.0,0,0
4,1,166,5,874965677,24,M,technician,85711,Manon of the Spring (Manon des sources) (1986),01-Jan-1986,...,0,0.0,0,0,0.0,0,0,0.0,0,0


In [3]:
data.columns

Index(['User ID', 'Item ID', 'Rating', 'timestamp', 'Age', 'Gender',
       'Occupation', 'zip code', 'Movie Title', 'Release Date', 'URL',
       'Unknown', 'Action', 'Adeventure', 'Animation', 'Childrens', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'],
      dtype='object')

Here User IDs identify people that rated different movies, Item IDs refer to the movies, and we have a range of information from ones about the user and ones that relate to the movie. 

We want to give movies they are likely to enjoy based on their past reviews.

## Matrix Factorisation

Our task is to create a collaborative filtering using matrix factorisation. 

Collaborative filtering is a model where based on movies it knows a user liked, it finds other users with similar ratings and proposes movies they liked as likely candidates. One possible approach would be to guess how a user would rate different movies, then recommend the ones we think they would rate highest. 

To do this we can use singular value decomposition to break down the ratings matrix $R$:

$$ R= U \Sigma V^\top $$

which contains the ratings each user gave to movies they watched, and 0 for ones they have not.

If we take a low-rank approximation of it:

$$ R_ k= U_k \Sigma_kV^\top_k $$

where we use only the k first singular values and vectors, the zeros where movies have not been rated will be smoothed out. By comparing these previously zero entries, we can expect the ones with the largest growth to respresent movies that would be highly rated.

By doing this we implicitly introduce k latent variables, where we can intrepret $U_k$ as a matrix describing how much a user likes those k attributes, and $V_k$ as a matrix showing how each movie is aligned with those k factors. Contextually this could combine things like genre or actors.

We start by creating the matrix of ratings.



In [4]:
ratings = data.pivot(index = 'User ID', columns ='Item ID', values = 'Rating').fillna(0) # sort by user and movie
ratings

Item ID,1,2,3,4,5,6,7,8,9,10,...,1660,1661,1662,1663,1664,1665,1679,1680,1681,1682
User ID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next to apply the SVD we centre the matrix.

In [5]:
r = ratings.values # converts to matrix
means = np.mean(r, axis = 1).reshape(-1,1) 
r2 = r - means

Now we can decompose the matrix $R$.

In [6]:
U1, sigma, Vt = svds(r2 , k =  100)
sigma = np.diag(sigma)

Then we multiply the matrices and add the means back to get our approximation.

In [7]:
p =((U1@sigma)@ Vt) + means
pred = pd.DataFrame(p,columns=ratings.columns)
pred.index += 1
pred.head()

Item ID,1,2,3,4,5,6,7,8,9,10,...,1660,1661,1662,1663,1664,1665,1679,1680,1681,1682
1,4.535338,3.377420,2.148937,4.209495,2.060415,2.719416,4.226810,0.967013,5.040234,2.782738,...,-0.010706,-0.051346,0.058949,0.022290,0.070588,0.022290,-0.010325,-0.013475,-0.022062,0.077854
2,3.799748,-0.219038,0.019976,0.403776,-0.116200,0.455710,0.036102,-0.228277,0.165513,1.874556,...,-0.027696,-0.029560,-0.013837,-0.019900,-0.006861,-0.019900,0.012220,0.000482,-0.052872,-0.056812
3,-0.073539,-0.066183,0.074263,-0.044759,0.027685,-0.100111,0.021347,-0.096357,0.006834,0.006971,...,0.016716,-0.001075,0.001984,-0.000089,0.002856,-0.000089,0.002446,0.000980,-0.001928,-0.011543
4,0.190695,-0.299708,0.048256,0.201174,-0.139472,0.009617,0.032057,-0.041062,-0.097846,-0.374609,...,0.038677,0.012169,-0.031160,-0.009319,-0.027393,-0.009319,0.023335,0.016914,-0.011643,-0.005425
5,4.231784,2.118404,0.209611,-0.322296,0.358826,0.134808,-0.476681,-0.359183,-0.056438,0.245643,...,0.044355,0.018810,-0.043988,-0.026660,-0.152268,-0.026660,-0.016105,-0.012787,-0.003800,-0.002031


To obtain recommendations we can select the highest ratings from each row:

In [8]:
def rec(UserID,k,mat):
    u = np.where(ratings.iloc[UserID]==0)[0] # find movies with rating 0 which mean they haven't been watched
    preds = pd.to_numeric(mat.iloc[UserID,u]).nlargest(k) # indices with highest k ratings
    index = preds.index
    titles = []
    for j in index:
        titles.append((data.loc[data['Item ID']==j])['Movie Title'].iloc[1]) # finds title of movie with index and add to list
    return pd.DataFrame({ 'Movie Title' : titles, 'Rating': preds})

And here are a few movies recommended for user 10:

In [9]:
rec(10,5,pred)

,Movie Title,Rating
Item ID,,
450,Star Trek V: The Final Frontier (1989),1.374115
444,"Blob, The (1958)",1.206311
226,Die Hard 2 (1990),1.145894
436,"American Werewolf in London, An (1981)",1.104973
193,"Right Stuff, The (1983)",1.061177


And this completes the SVD algorithm.

## Gradient Descent

With SVD we created a number of latent variables which include information about the movies. Looking at the data however, their genres are available to us as attributes that were completely ignored. It would be natural to consider taking them as the latent factors in the factorisation. Since there are only 18 genres which may not necessarily encode all information on every movie, against 50 latent variables used previously, it is unknown whether this would give better results. However we would at leasst gain more interpretative insight through having variables we recognise. 

We start by collecting the movies and genres.

In [10]:
dropped = data.drop_duplicates(subset='Item ID') # drop extra appearances of each movie in data
dropped = dropped[['Item ID','Action','Adeventure', 'Animation', 'Childrens', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western']] # take all genres
dropped = dropped.set_index('Item ID') # use movie ID as index
dropped[dropped != 1] = 0 # remove invalid values
dropped = dropped.sort_index() 
dropped.head()

,Action,Adeventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
Item ID,,,,,,,,,,,,,,,,,,
1,0.0,0,0,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
2,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0
3,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0
4,1.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0
5,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,1.0,0,0


Here similar to SVD we have 

$$ R = UV^\top $$

There are 943 users, 605 movies, and 18 genres, so $R$ is $943 \times 605$, $U$ is $943 \times 18$, and $V$ is $605 \times 18$. But unlike SVD we do not have an immediate decomposition, so we resort to gradient descent.

At each time step we compute the estimate

$$ \hat{R} = U_n V_n^\top$$

This allows us to compute the mean squared error by summing over $(R-\hat{R})^2$, and it can be shown that the update equation for U is given by 

$$\begin{equation} 
\begin{split}
U & = U - \alpha \frac{\partial MSE}{\partial U} \\
 & = U - \alpha [\frac{-2}{N}(R-\hat{R})V]
\end{split}
\end{equation}$$

with some learning parameter $\alpha$ and $N$ being the number of entries in $R$.

We have an analogous equation for $V$. However, unlike typical algorithms, here the matrix $V$ is simply the matrix of genres which we know as a ground truth. Hence there is no sense in updating it and we can hold it constant while optimising $U$ only.

In [11]:
U = np.random.uniform(-1, 1, size=(943, 18)) # initialise U randomly 

In [12]:
def descent(U,steps,alpha):
    preds = U@np.transpose(dropped) # initial estimate of R
    preds.index += 1 # default index starts from 0 but Item IDs start from 1
    N = 943 * 605
    while steps > 0:
        steps -= 1
        dU = np.matmul(-2/N * (ratings - preds),dropped) 
        U -= alpha * dU 
        preds = np.matmul(U,np.transpose(dropped))
    return U

Now we can approximate $U$ and multiply it with $V$ to get the final result. (May take a minute to run)

In [13]:
U = descent(U,100,0.5)
pred2 = np.matmul(U,np.transpose(dropped)) # predicted ratings

Once converged the matrix $U$ suggests each user's perferences towards different genres as shown below. 

In [14]:
U.head()

,Action,Adeventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
User ID,,,,,,,,,,,,,,,,,,
1,-0.426104,-0.614734,0.493021,0.837858,-0.582243,-0.465019,0.607041,0.477177,-0.39967,0.594343,-0.303864,-0.129623,-0.689394,0.150715,-0.536912,0.984779,-0.769811,0.471061
2,0.097835,0.441261,0.252283,-0.089647,-0.763975,0.063446,0.542152,0.012614,0.23039,-0.544454,-0.199647,-0.085813,-0.306167,-0.942438,0.062721,0.859645,0.563211,-0.365516
3,-0.309321,-0.533474,-0.872316,0.019895,-0.944574,0.231818,-0.286244,0.591306,0.247824,-0.550233,0.488525,0.749423,-0.599207,-0.846311,-0.581722,-0.024808,-0.714123,0.24868
4,0.870115,0.693214,0.205984,0.753163,-0.396843,0.985653,0.244908,0.128712,-0.204969,-0.622422,-0.248991,0.38704,0.034523,0.044914,0.114006,-0.296536,0.153869,-0.341539
5,-0.781955,0.7657,-0.208805,0.835786,0.787744,-0.899033,0.883887,0.456906,-0.868845,-0.368417,0.197787,0.35906,-0.363662,0.627759,-0.707523,-0.138035,-0.95713,0.490867


And with the predicted ratings we can run the recommendations as before: (sometimes gives an error, try running all cells starting from generating $U$)

In [15]:
rec(10,5,pred2)

,Movie Title,Rating
Item ID,,
855,Diva (1981),1.219210
1003,That Darn Cat! (1997),0.937451
1444,That Darn Cat! (1965),0.937451
644,"Thin Blue Line, The (1988)",0.927892
645,Paris Is Burning (1990),0.927892


## Model comparison

We decided to use the root mean squared error as the performance metric. This uses unseen test data with new ratings that we can compare to the predicted ratings. 

In [16]:
test = pd.read_csv('user_test_df.csv')
test.head()

,User ID,Item ID,Rating,timestamp,Age,Gender,Occupation,zip code,Movie Title,Release Date,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,201,3,878542960,24,M,technician,85711,Evil Dead II (1987),01-Jan-1987,...,0,0.0,1,0,0.0,0,0,0.0,0,0
1,1,12,5,878542960,24,M,technician,85711,"Usual Suspects, The (1995)",14-Aug-1995,...,0,0.0,0,0,0.0,0,0,1.0,0,0
2,1,208,5,878542960,24,M,technician,85711,Young Frankenstein (1974),01-Jan-1974,...,0,0.0,1,0,0.0,0,0,0.0,0,0
3,1,3,4,878542960,24,M,technician,85711,Four Rooms (1995),01-Jan-1995,...,0,0.0,0,0,0.0,0,0,1.0,0,0
4,1,241,4,878543133,24,M,technician,85711,"Last of the Mohicans, The (1992)",01-Jan-1992,...,0,0.0,0,0,0.0,1,0,0.0,1,0


In [17]:
rt = test.pivot(index = 'User ID', columns ='Item ID', values = 'Rating').fillna(0)
print(rt.shape)
rt.head()

(943, 496)


Item ID,1,2,3,4,5,6,7,8,9,10,...,1650,1651,1652,1658,1659,1661,1662,1664,1665,1682
User ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,0.0,3.0,5.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We see the number of columns has decreased, meaning some movies were not rated in the test set and were excluded. This made vectorisation difficult as the non zero entries in each row could not be directly subtracted from the predicted values with the same indices. Hence we resort to for loops. 

In [18]:
error = 0
for i in range(943):
    for j in range(496):
        if rt.iloc[i,j] != 0:
            itemid = rt.columns[j] 
            error = error + (rt.iloc[i,j] - pred.loc[i+1,itemid])**2 # match with predicted ratings with correct indices

In [19]:
terms = sum([len(np.nonzero(rt.loc[i])[0]) for i in range(1,944)])
(error/terms)**0.5

np.float64(1.1916326461134061)

## Conclusion

In this report we used matrix factorisation to create a collaborative filtering to recommend movies. We explored SVD and gradient descent as possible methods and produced predicted ratings from which we took the top scoring movies. The two have different predictive and interpretative properties that can be chosen to fit different practical needs.

## References

https://beckernick.github.io/matrix-factorization-recommender/

https://sparkbyexamples.com/pandas/pandas-iloc-usage-with-examples/

https://stackoverflow.com/questions/13070461/get-indices-of-the-top-n-values-of-a-list

https://discuss.datasciencedojo.com/t/how-to-find-the-row-number-of-nth-largest-value/984/4

https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/03_subset_data.html

https://medium.com/@maxbrenner-ai/matrix-factorization-for-collaborative-filtering-linear-to-non-linear-models-in-python-5cf54363a03c